In [1]:
import requests
import lxml.html
import pandas as pd
import sqlite3
from pandas.io import sql
import os
import time

In [10]:
def db_save(NEWS_LIST):
    with sqlite3.connect(os.path.join('.', 'sqliteDB')) as con:
        try:
            NEWS_LIST.to_sql(name = 'NEWS_LIST', con = con, index = False, if_exists = 'append')
        except Exception as e:
            print(str(e))
        print(len(NEWS_LIST), '건 저장완료...')

In [11]:
def db_delete():
    with sqlite3.connect(os.path.join('.','sqliteDB')) as con: 
        try:
            cur = con.cursor()
            sql = 'DELETE FROM NEWS_LIST'
            cur.execute(sql)
        except Exception as e:
            print(str(e)) 

In [12]:
def db_select():
    with sqlite3.connect(os.path.join('.','sqliteDB')) as con: 
        try:
            query = 'SELECT * FROM NEWS_LIST'
            NEWS_LIST = pd.read_sql(query, con = con)
        except Exception as e:
            print(str(e)) 
        return NEWS_LIST

In [13]:
import re
import string

def get_detail(url):
    body = []
    punc = '[!"#$%&\'()*+,-./:;<=>?[\]^_`{|}~“”·]'
    response = requests.get(url)
    root = lxml.html.fromstring(response.content)
    for p in root.xpath('//*[@id="harmonyContainer"]/section/p'):
        if p.text: # 체크
            body.append(re.sub(punc, '', p.text)) # 특수문자 제거
    full_body = ' '.join(body)
    
    return full_body

In [14]:
page = 58
max_page = 0
REG_DATE = '20210712'

In [15]:
while(True):
    df_list = []
    response = requests.get('http://news.daum.net/breakingnews/digital?page={}&regDate={}'\
                            .format(page, REG_DATE))
    root = lxml.html.fromstring(response.content)
    for li in root.xpath('//*[@id="mArticle"]/div[3]/ul/li'):
        a = li.xpath('div/strong/a')[0]
        url = a.get('href')
        article = get_detail(url)
        df = pd.DataFrame({'URL' : [url],'TITLE':[a.text],'ARTICLE' : [article]})
        df_list.append(df)   
        
    if df_list:   
        df_10 = pd.concat(df_list)
        db_save(df_10)

    # 페이지 번호 중에서 max 페이지 가져오기    
    for a in root.xpath('//*[@id="mArticle"]/div[3]/div/span/a'):
        try:
            num = int(a.text)
            if max_page < num:
                max_page = num       
        except:
            pass

    # 마지막 페이지 여부 확인     
    span = root.xpath('//*[@id="mArticle"]/div[3]/div/span/a[@class="btn_page btn_next"]')

    if (len(span) <= 0) & (page > max_page):
        break
    else:
        page = page + 1
        
    time.sleep(1)      

15 건 저장완료...
15 건 저장완료...
15 건 저장완료...
15 건 저장완료...
8 건 저장완료...
8 건 저장완료...


In [16]:
print(db_select())

                                       URL  \
0   https://v.daum.net/v/20210712082450370   
1   https://v.daum.net/v/20210712082351344   
2   https://v.daum.net/v/20210712082058258   
3   https://v.daum.net/v/20210712082032251   
4   https://v.daum.net/v/20210712082008239   
..                                     ...   
71  https://v.daum.net/v/20210712014601054   
72  https://v.daum.net/v/20210712000421416   
73  https://v.daum.net/v/20210712000107369   
74  https://v.daum.net/v/20210712000104363   
75  https://v.daum.net/v/20210712000007329   

                                     TITLE  \
0                  1만배 빠른 반도체 소자 인쇄 새공법 개발   
1           대만 폭스콘·TSMC, 코로나19 백신 1천만회분 구매   
2               英억만장자 리처드 브랜슨, 첫 우주관광비행 성공   
3   벨기에서 알파·베타 변이 '복합 감염' 사망자 나왔다..세계 첫 사례   
4                 비트코인 4000만원대..이더리움 250만원   
..                                     ...   
71                억만장자 브랜슨, 준궤도 우주관광 1호 됐다   
72        [Biz & Now] LG유플러스, 5G 스마트팩토리 조성   
73            쿠버네티스 플랫폼에 최적화된 데이터